In [14]:
import pandas as pd
import numpy as np

# define the name of the csv file you want to read in
accounts = './data/accounts.csv'
playbacks = './data/playbacks.csv'
subscriptions = './data/subscriptions.csv'

# PLAYBACKS

In [15]:
# Read playbacks
df_playbacks = pd.read_csv(playbacks)
df_playbacks.columns = df_playbacks.columns.str.lower()
display(df_playbacks.shape)
display(df_playbacks.date_start.tail())
# one account can have multiple subscriptions
# what about pausing the movie
# IP + User_Agent-> check for multiple persons per subscriptions
    # IP/Account
# average playback duration
# cancelled session?

(139569, 7)

139564    2020-10-21 13:21:30
139565    2020-10-21 12:35:04
139566    2020-10-21 12:05:14
139567    2020-10-21 11:46:11
139568    2020-10-21 11:18:58
Name: date_start, dtype: object

In [16]:
df_playbacks.isna().sum()

subscription_key    0
account_key         0
movie_id            0
date_start          0
playback_time       0
user_agent          0
ip_hash             0
dtype: int64

In [17]:
df_playbacks.describe()
# have a closer look at max playback times

,movie_id,playback_time
count,139569.000000,139569.00000
mean,712.801589,6442.57679
std,346.698325,8191.70516
min,8.000000,960.00000
25%,453.000000,4680.00000
50%,745.000000,5760.00000
75%,977.000000,6660.00000
max,1344.000000,472920.00000


In [19]:
#DIFFERENTIATION BETWEEN DESKTOP/MOBILE
#TODO Create 'DESKTOP vs. MOBILE' column
#TODO filmingo app users dedicated column?!

#df_playbacks.user_agent.count()
#5387 unique values
#139.569 count

#SEARCH FOR 'Macintosh'
#df_playbacks[df_playbacks.user_agent.str.contains('Macintosh', na=False)].user_agent.count()
#64.401 count ~50%

#SEARCH FOR 'Windows'
#df_playbacks[df_playbacks.user_agent.str.contains('Windows', na=False)].user_agent.count()
#41.475

#DESKTOP Views
df_playbacks[df_playbacks.user_agent.str.contains(('Windows|Macintosh|TV|Linux'), na=False)].user_agent.count()
#117.081 Desktop playbacks

# Mobile Views
#df_playbacks[df_playbacks.user_agent.str.contains(('Android|iOS|iPhone|iPad'), na=False)].user_agent.count()
# 22.800 Mobile views

df_playbacks[df_playbacks.user_agent.str.contains(('Linux'), na=False)].user_agent.unique()
df_playbacks[df_playbacks.user_agent.str.contains(('Darwin'), na=False)].user_agent.count()

#HOW MANY PEOPLE ARE USING THE APP
df_playbacks[~df_playbacks.user_agent.str.contains('filmingo', na=False)].user_agent.count()
#13.173 playbacks via the app



126396

In [27]:
#Let's create the new columns
df_playbacks.loc[df_playbacks['user_agent'].str.contains('Windows|Macintosh|TV|Linux|Darwin|CrOS'), 'device'] = 'desktop'
df_playbacks.loc[df_playbacks['user_agent'].str.contains('Android|iOS|iPhone|iPad'), 'device'] = 'mobile'
df_playbacks.loc[df_playbacks['user_agent'].str.contains('filmingo'), 'app user'] = 'yes'
df_playbacks.loc[~df_playbacks['user_agent'].str.contains('filmingo'), 'app user'] = 'no'
df_playbacks.groupby('device').count()
#df_playbacks[~df_playbacks.device.str.contains(('Android|iOS|iPhone|iPad|Windows|Macintosh|TV|Linux'), na=False)].user_agent.count()



,subscription_key,account_key,movie_id,date_start,playback_time,user_agent,ip_hash,app user
device,,,,,,,,
desktop,116734,116734,116734,116734,116734,116734,116734,116734
mobile,22800,22800,22800,22800,22800,22800,22800,22800


In [28]:
df_playbacks.head()

,subscription_key,account_key,movie_id,date_start,playback_time,user_agent,ip_hash,device,app user
0,9679626b6cc1765f8bc8da98fda2d8e8f28f40736fd777...,024ebde74e23ca924cff0a3989e7f87efe2b2cbcd7ad2c...,1189,2022-09-30 23:55:31,12420,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,0b14aa31d5635ee899be928089fd810c655a7ec7785911...,desktop,no
1,513d006472c99ef047c64bafad57a39906ad5c618505b1...,8968e3851322a89c265a0ec4c81341361e3b0953502d65...,1327,2022-09-30 23:48:36,1440,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4...,c65100e7d48b9e06ca6d89ffe594066553ea63ddc70ee1...,desktop,no
2,cac0c77d3aedc55303b8dfea314052eba5dcf4817e4962...,8a2d7ac64595778ed5fe7edb2dd43f843f9879d773df1a...,212,2022-09-30 23:47:33,1080,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...,db17d38bf724c377dcd463f7ad8df8279b6122c8890773...,desktop,no
3,a35503448084642ac8651a0f482db410ccdd7c7716df1d...,7f654ed491a8d1c5b6eb54ea98f6b30b0f9d9515aaad22...,108,2022-09-30 23:46:52,5400,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...,69839531a94bf2468744c083a7bf4b78f39c8b1b299c34...,desktop,no
4,ff97a5e4ea0bb411f72e2e1750fb94b248ec2026406a3f...,6efdbec56858f25686797c974bfa67809a718408bd3631...,1314,2022-09-30 23:16:49,10380,"filmingo v2.1 (83); iOS 15.6.1; iPad8,11",652d233c8526ebefb7a8e939ca546fe1a76498775b682a...,mobile,yes


# EDA part

In [29]:
import sql_functions as sf
schema = 'capstone_filmingo'

In [31]:
sf.get_dataframe(f' SELECT DISTINCT * FROM {schema}.playbacks WHERE device IS NULL')



,playback_ID,subscription_key,account_key,movie_id,date_start,playback_time,user_agent,ip_hash,device,app user
0,907,fb2772f47cb0c9c503d2c1f0d805fa27782f2d2ba7d2b5...,6a597183708020f5909c9d8e9525967ce602a394fbbcb5...,738,2020-10-24 21:36:54,5940,Mozilla/5.0 (X11; CrOS x86_64 13421.53.0) Appl...,497b8d467ee1a552bac1ebdc2de290525d8fcef8e4f384...,None,no
1,1235,fb2772f47cb0c9c503d2c1f0d805fa27782f2d2ba7d2b5...,6a597183708020f5909c9d8e9525967ce602a394fbbcb5...,602,2020-10-25 21:44:59,1620,Mozilla/5.0 (X11; CrOS x86_64 13421.53.0) Appl...,497b8d467ee1a552bac1ebdc2de290525d8fcef8e4f384...,None,no
2,4242,7625797fc15744dcf3f42c8c47d45053c5405a5fb22475...,7460a75f94a60876c7cbacd626f7e438d1c2e4b9b974fa...,35,2020-11-11 12:34:01,2340,Mozilla/5.0 (X11; CrOS x86_64 13421.73.0) Appl...,b1052576719d5364d142ee96420cbbacb811416f9d6db3...,None,no
3,4863,243584eeb68b8425583c614c85efd9ceea6e2e525cbce3...,66a4de6abe97c652f5f4a8a8897d4af537baf047da0eb1...,540,2020-11-14 20:15:35,5760,Mozilla/5.0 (X11; CrOS x86_64 12239.92.4) Appl...,d52c0d8b929f863bd12fb55d249d2d381a897f2ddd3ab7...,None,no
4,5226,fb2772f47cb0c9c503d2c1f0d805fa27782f2d2ba7d2b5...,6a597183708020f5909c9d8e9525967ce602a394fbbcb5...,612,2020-11-15 22:13:35,6480,Mozilla/5.0 (X11; CrOS x86_64 13421.89.0) Appl...,7215d98f0427a8d71fffe8e0bd314830e9ccddc83eabb9...,None,no
...,...,...,...,...,...,...,...,...,...,...
190,133186,f10a004b565035a2eb44b414a541d2c384f6f7b986191b...,52c234479d53c2c2e2fe4ea751830d0bfc6da7c9e25344...,773,2022-08-14 18:07:30,1620,Mozilla/5.0 (X11; CrOS aarch64 14816.131.0) Ap...,c2dfc1db5a85bb04fe5e8e691639603a0d005304318286...,None,no
191,133701,f10a004b565035a2eb44b414a541d2c384f6f7b986191b...,52c234479d53c2c2e2fe4ea751830d0bfc6da7c9e25344...,1263,2022-08-18 18:57:48,7380,Mozilla/5.0 (X11; CrOS aarch64 14909.100.0) Ap...,3aac1e35b1c67d1f8f3673049e08e08db6eba2aa94a939...,None,no
192,136799,daebbb70f06cfc1ceef4a6b8c1a1e0c3d3b8df5958a3fb...,2a3a30ccbb0769d42bc7db09d28a73be9e19261642f40d...,199,2022-09-10 22:13:49,3240,Mozilla/5.0 (X11; CrOS x86_64 14909.132.0) App...,49a081a2fa2c5754c920b250b66d12e79398b6581d365f...,None,no
193,137180,daebbb70f06cfc1ceef4a6b8c1a1e0c3d3b8df5958a3fb...,2a3a30ccbb0769d42bc7db09d28a73be9e19261642f40d...,1223,2022-09-14 19:35:55,2160,Mozilla/5.0 (X11; CrOS x86_64 14989.85.0) Appl...,bbbe84dbbea652700c82691619e247275d71169771646e...,None,no
